<div style="text-align:center;font-size:22pt; font-weight:bold;color:white;border:solid black 1.5pt;background-color:#1e7263;">
    TensorBoard Callback: Breast Cancer Project
</div>

In [1]:
# ======================================================================= #
# Course: Deep Learning Complete Course (CS-501)
# Author: Dr. Saad Laouadi
# Institution: Quant Coding Versity Academy
# Date: December 25, 2024
#
# ==========================================================
# Lesson: Learning Rate Adaptation with ReduceLROnPlateau
#         Synthetic Data Example
# ==========================================================
# ## Learning Objectives
# This example will enable you to:
# 1. Create synthetic data for learning rate adaptation
# 2. Implement ReduceLROnPlateau callback
# 3. Monitor learning rate changes during training
# 4. Visualize the impact of learning rate reduction
# 5. Compare training with and without adaptive learning rates
# =======================================================================
#          Copyright © Dr. Saad Laouadi 2024
# =======================================================================

In [4]:
# ============================================================================ #
#                         Environment Path Configuration                       #
# ============================================================================ #
#
# Purpose:
#   Configure the system PATH to use Python executables from the active virtual 
#   environment instead of global installations.
#
# Usage:
#   1. First verify if configuration is needed by running: !which python
#   2. If the output shows the global Python installation rather than your 
#      virtual environment, execute this configuration block
#
# Note:
#   This configuration is typically only needed for JupyterLab Desktop or 
#   similar standalone installations. Web-based JupyterLab or properly 
#   configured environments should not require this adjustment.
# ============================================================================ #

import os
import sys

env_path = os.path.dirname(sys.executable)
os.environ['PATH'] = f"{env_path}:{os.environ['PATH']}"

In [5]:
# ==================================================== #
#        Load Required Libraries
# ==================================================== #
import shutil
from datetime import datetime
import io

# Disable Metal API Validation
os.environ["METAL_DEVICE_WRAPPER_TYPE"] = "0"  

import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


# import tensorflow
import tensorflow as tf

# Set styling for better visualization
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("="*72)

%reload_ext watermark
%watermark -a "Dr. Saad Laouadi" -u -d -m

print("="*72)
print("Imported Packages and Their Versions:")
print("="*72)

%watermark -iv
print("="*72)

# Global Config
RANDOM_STATE = 101

Author: Dr. Saad Laouadi

Last updated: 2025-01-09

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 24.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Imported Packages and Their Versions:
numpy     : 1.26.4
matplotlib: 3.9.2
seaborn   : 0.13.2
psutil    : 5.9.0
sys       : 3.11.10 (main, Oct  3 2024, 02:26:51) [Clang 14.0.6 ]
pandas    : 2.2.2
sklearn   : 1.5.1
tensorflow: 2.16.2



In [6]:
# =============================================================================
# Educational Utility Function: Directory Cleanup
# =============================================================================
#
# Purpose:
#   This utility function is designed for educational purposes in the context
#   of programming tutorials and exercises. It provides a safe way to clean up
#   working directories during course activities.
#
# Usage Context:
#   - Instructors: Useful for preparing and resetting educational content
#   - Students: Helpful when experimenting with code examples
#   - Learning Environment: Supports clean workspace management
#
# Note:
#   This function is intended for educational environments only and should be
#   used with caution as it permanently deletes directory contents.
# =============================================================================

def cleanup_directory(directory_path):
    """
    Deletes the specified directory and all its contents.
    Args:
        directory_path (str): Path to the directory to delete.
    """
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        shutil.rmtree(directory_path)
        print(f"Directory '{os.path.basename(directory_path)}' deleted successfully.")
    else:
        print(f"Directory '{os.path.basename(directory_path)}' does not exist or is not a directory.")
        
        
# tensorboard cleanup        
def cleanup_tensorboard():
    """
    Cleanup TensorBoard processes professionally
    """
    # Find and terminate TensorBoard processes
    for proc in psutil.process_iter(['pid', 'name']):
        try:
            # Look for tensorboard processes
            if 'tensorboard' in proc.info['name'].lower():
                # Terminate gracefully
                process = psutil.Process(proc.info['pid'])
                process.terminate()
                print(f"TensorBoard process {proc.info['pid']} terminated gracefully")
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass

---

### Requirements

This notebook needs the following tools to run without errors. You can install the next tools from within a notebook cell without leaving this working environment. 

I recommend running this first

```python
import os
import sys

env_path = os.path.dirname(sys.executable)
os.environ['PATH'] = f"{env_path}:{os.environ['PATH']}"
```

Check the cell {[2]} above for more information. 


1. **The `psutil` package**: You might need to install `psutil` if you haven't already by running the next command:

```python
!pip install psutil
```

2. **The `tensorboard` package**: You may install `tensorboard` by running this command

```bash
!pip install tensorboard
```

If the somethind did not go as expected, you may need to check this [tensorboard overview](07-04-00-callbacks-tensorboard-overview.ipynb) for more details on to setup your environment to use `tensorboard`.

### Raodmap
1. Load and process Breast Canser Dataset
2. Create Model with `tensorboard` callback
4. Use and interpret the results on tensorboard visualization tool
5. Use more advanced model with multiple callbacks.

In [7]:
# ================================================================= #
#          Breast Cancer Project
# ================================================================= #

def load_and_preprocess_data(random_state=0):
    """Load and preprocess the Breast Cancer dataset"""
    # Load data
    cancer = load_breast_cancer()
    X, y = cancer.data, cancer.target
    
    # Create feature names DataFrame for better understanding
    feature_df = pd.DataFrame(X, columns=cancer.feature_names)
    
    # Split data first with original labels
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=0.2, 
        random_state=random_state, 
        stratify=y                 # stratification
    )
    
    # Then convert labels to one-hot encoding
    y_train = tf.keras.utils.to_categorical(y_train)
    y_test = tf.keras.utils.to_categorical(y_test)
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    return X_train_scaled, X_test_scaled, y_train, y_test, cancer.feature_names

# The model function
def create_model(input_shape, initial_lr=0.001):
    """Create a model with specified initial learning rate"""
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.1),
        
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        
        tf.keras.layers.Dense(2, activation='softmax')  
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=initial_lr),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=[
            tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
            tf.keras.metrics.AUC(name='auc'),
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall')
        ]
    )
    
    return model


# Set up callbacks
def create_callbacks(model_name="breast_cancer"):
    """Create all callbacks for training"""
    
    # Create directories for logs and checkpoints
    base_dir = "training_logs"
    log_dir = os.path.join(base_dir, "logs", model_name, datetime.now().strftime('%Y%m%d-%H%M%S'))
    checkpoint_dir = os.path.join(base_dir, "checkpoints", model_name)
    os.makedirs(log_dir, exist_ok=True)
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    callbacks = [
        # TensorBoard callback
        tf.keras.callbacks.TensorBoard(
            log_dir=log_dir,
            histogram_freq=1,
            write_graph=True,
            write_images=False,
            update_freq='epoch',
            profile_batch=(5, 10)
        ),
        
        # Model checkpoint callback
        tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(checkpoint_dir, 'model_{epoch:02d}-{val_loss:.4f}.keras'),
            save_best_only=True,
            monitor='val_loss',
            mode='min',
            verbose=0
        ),
        
        # Early stopping callback
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=0
        ),
        
        # Reduce LR on plateau callback
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            min_lr=1e-6,
            verbose=0
        )
    ]
    
    return callbacks

In [10]:
# Clean up previous TensorBoard sessions
%reload_ext tensorboard
!rm -rf training_logs/logs/*

In [11]:
# Load and preprocess data
X_train, X_test, y_train, y_test, feature_names = load_and_preprocess_data()

# Create and train the model
model = create_model((X_train.shape[1],))
callbacks = create_callbacks()

In [12]:
%timeit
print("start trarining")
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=callbacks,
    verbose=0
)

print("training finished...!")

start trarining
training finished...!


In [13]:
# Launch TensorBoard
%tensorboard --logdir training_logs/logs

Reusing TensorBoard on port 6007 (pid 4661), started 2:23:30 ago. (Use '!kill 4661' to kill it.)

In [14]:
def setup_tensorboard_with_figure_saving(model_name="test_tensorboard"):
    """
    Set up TensorBoard callback with figure saving capability
    """
    log_dir = os.path.join("logs", "fit", model_name, 
                          datetime.now().strftime('%Y%m%d-%H%M%S'))
    os.makedirs(log_dir, exist_ok=True)
    
    # Create file writer
    file_writer = tf.summary.create_file_writer(log_dir)
    
    # Function to save figures
    def plot_to_tensorboard(epoch, logs):
        with file_writer.as_default():
            # Create and save loss plot
            plt.figure(figsize=(10, 6))
            plt.plot(logs['loss'], label='Training Loss')
            plt.plot(logs['val_loss'], label='Validation Loss')
            plt.title('Model Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.legend()
            
            # Convert plot to image
            buf = io.BytesIO()
            plt.savefig(buf, format='png')
            plt.close()
            
            # Write to TensorBoard
            image = tf.image.decode_png(buf.getvalue(), channels=4)
            image = tf.expand_dims(image, 0)
            tf.summary.image("Training Loss Plot", image, step=epoch)
    
    # Create TensorBoard callback
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=1,
        write_graph=True,
        write_images=True,
        update_freq='epoch',
        profile_batch=(5, 10)
    )
    
    # Create custom callback for saving plots
    plot_callback = tf.keras.callbacks.LambdaCallback(
        on_epoch_end=plot_to_tensorboard
    )
    
    return [tensorboard_callback, plot_callback]

# Use in your training
callbacks = setup_tensorboard_with_figure_saving()
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.9886 - auc: 0.9973 - loss: 0.0570 - precision: 0.9886 - recall: 0.9886 - val_accuracy: 0.9890 - val_auc: 0.9999 - val_loss: 0.0380 - val_precision: 0.9890 - val_recall: 0.9890
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.9643 - auc: 0.9963 - loss: 0.0839 - precision: 0.9643 - recall: 0.9643 - val_accuracy: 0.9890 - val_auc: 0.9999 - val_loss: 0.0374 - val_precision: 0.9890 - val_recall: 0.9890
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.9802 - auc: 0.9964 - loss: 0.0686 - precision: 0.9802 - recall: 0.9802 - val_accuracy: 0.9890 - val_auc: 0.9999 - val_loss: 0.0378 - val_precision: 0.9890 - val_recall: 0.9890
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.9839 - auc: 0.9956 - loss: 0.0734 - precision: 0.9839 - recall: 0.9839 - val_accuracy: 0.9890 - val_auc: 0.9999 - val_loss: 0.0383 - val_precision: 0.9890 - val_recall: 0.9890
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━

In [19]:
# Launch TensorBoard
%tensorboard --logdir training_logs/logs

Reusing TensorBoard on port 6007 (pid 4661), started 2:31:28 ago. (Use '!kill 4661' to kill it.)

In [20]:
from tensorboard.plugins.hparams import api as hp

In [17]:
def save_training_plots(history, save_dir="tensorboard_plots"):
    """
    Save training history plots to a specified directory
    """
    # Create directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)
    
    # Get list of metrics from history
    metrics = [metric for metric in history.history.keys() 
              if not metric.startswith('val_')]
    
    # Plot each metric
    for metric in metrics:
        plt.figure(figsize=(10, 6))
        
        # Plot training metric
        plt.plot(
            history.history[metric],
            label=f'Training {metric}'
        )
        
        # Plot validation metric if it exists
        if f'val_{metric}' in history.history:
            plt.plot(
                history.history[f'val_{metric}'],
                label=f'Validation {metric}'
            )
            
        plt.title(f'Model {metric}')
        plt.xlabel('Epoch')
        plt.ylabel(metric)
        plt.legend()
        plt.grid(True)
        
        # Save the plot
        plt.savefig(os.path.join(save_dir, f'{metric}_plot.png'))
        plt.close()
    
    print(f"Plots saved in directory: {save_dir}")

In [18]:

# Use in your training
callbacks = setup_tensorboard_with_figure_saving()
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=callbacks,
    verbose=2
)

# Use after model training
save_training_plots(history)

Epoch 1/50
12/12 - 1s - 53ms/step - accuracy: 0.9808 - auc: 0.9925 - loss: 0.0853 - precision: 0.9808 - recall: 0.9808 - val_accuracy: 0.9890 - val_auc: 0.9996 - val_loss: 0.0385 - val_precision: 0.9890 - val_recall: 0.9890
Epoch 2/50
12/12 - 1s - 49ms/step - accuracy: 0.9890 - auc: 0.9986 - loss: 0.0511 - precision: 0.9890 - recall: 0.9890 - val_accuracy: 0.9890 - val_auc: 0.9996 - val_loss: 0.0383 - val_precision: 0.9890 - val_recall: 0.9890
Epoch 3/50
12/12 - 1s - 49ms/step - accuracy: 0.9835 - auc: 0.9984 - loss: 0.0493 - precision: 0.9835 - recall: 0.9835 - val_accuracy: 0.9890 - val_auc: 0.9996 - val_loss: 0.0380 - val_precision: 0.9890 - val_recall: 0.9890
Epoch 4/50
12/12 - 1s - 51ms/step - accuracy: 0.9890 - auc: 0.9986 - loss: 0.0455 - precision: 0.9890 - recall: 0.9890 - val_accuracy: 0.9890 - val_auc: 0.9996 - val_loss: 0.0387 - val_precision: 0.9890 - val_recall: 0.9890
Epoch 5/50
12/12 - 1s - 50ms/step - accuracy: 0.9780 - auc: 0.9935 - loss: 0.0901 - precision: 0.9780 - 